In [118]:
import numpy as np
import random
from matplotlib import pyplot as plt
from matplotlib import animation

In [131]:
class particle_array():
    def __init__(self, n, dims, seed):
        self.g = random.seed(seed)
        self.lbound = dims[0]
        self.ubound = dims[1]
        self.llimit = dims[2]
        self.ulimit = dims[3]
        self.particles = [particle([random.randrange(self.lbound, self.ubound),
                                    random.randrange(self.lbound, self.ubound),
                                    random.randrange(self.lbound, self.ubound)], 
                                   [0, 0, 0], 
                                   self.llimit, 
                                   self.ulimit) for i in range(n)]
        for particle_1 in self.particles:
            total = 0
            for particle_2 in self.particles:
                if id(particle_1) != id(particle_2):
                    total += particle_2.m * pairwise_calculator(particle_1, particle_2).gaus_kern()
            particle_1.initrho = total
            particle_1.rho = total
            particle_1.initpre = (100 * particle_1.initrho * (particle_1.vmax ** 2))/7
            particle_1.pre = (100 * particle_1.initrho * (particle_1.vmax ** 2))/7
        self.calc_accel()
        self.calc_veloc()
            
    def get_position(self):
        for particle in self.particles:
            print(particle.p)
    def get_direction(self):
        for particle in self.particles:
            print(particle.d)
    def get_initrho(self):
        for particle in self.particles:
            print(particle.initrho)
    def get_initpre(self):
        for particle in self.particles:
            print(particle.initpre)
    def get_rho(self):
        for particle in self.particles:
            print(particle.rho)
    def get_pre(self):
        for particle in self.particles:
            print(particle.pre)
    def get_accel(self):
        for particle in self.particles:
            print(particle.a)
    def get_veloc(self):
        for particle in self.particles:
            print(particle.d)
            
    def calc_accel(self):
        for particle_1 in self.particles:
            total = 0
            for particle_2 in self.particles:
                if id(particle_1) != id(particle_2):
                    total += (particle_2.m)*\
                    ((particle_1.pre / (particle_1.rho ** 2)) + (particle_2.pre / (particle_2.rho ** 2))) *\
                    (pairwise_calculator(particle_1, particle_2).dgaus_kern())
            particle_1.a = -total
            
    def calc_veloc(self):
        for particle in self.particles:
            for i in range(3):
                if i == 2:
                    potential_speed = (particle.dt * particle.a) - (particle.dt * particle.g)
                    if potential_speed + particle.d[i] > particle.vmax:
                        particle.d[i] = particle.vmax
                    elif potential_speed + particle.d[i] < -particle.vmax:
                        particle.d[i] = -particle.vmax
                    else:
                        particle.d[i] += potential_speed
                else:
                    potential_speed = (particle.dt * particle.a)
                    if potential_speed + particle.d[i] > particle.vmax:
                        particle.d[i] = particle.vmax
                    elif potential_speed + particle.d[i] < -particle.vmax:
                        particle.d[i] = -particle.vmax
                    else:
                        particle.d[i] += potential_speed
                        
    def move(self):
        for particle in self.particles:
            

In [132]:
class particle():
    def __init__(self, position, direction, llimit, ulimit):
        self.p = position
        self.d = direction
        self.llimit = llimit
        self.ulimit = ulimit
        self.m = 1
        self.dt = 0.01 #s
        self.initrho = 0
        self.rho = 0
        self.initpre = 0
        self.pre = 0
        self.vmax = 30
        self.a = 0
        self.g = 9.81
    
    def get_magnitude(self):
        x, y, z = self.p
        return ((x**2) + (y**2) + (z**2))**(1/2)
        #return np.linalg.norm(self.p) This one works too

In [133]:
class pairwise_calculator():
    def __init__(self, particle1, particle2):
        self.p1 = particle1
        self.p2 = particle2
        
    def gaus_kern(self, h = 1, d = 3):
        mag_1 = self.p1.get_magnitude()
        mag_2 = self.p2.get_magnitude()
        delta_mag = mag_1 - mag_2
        return ((1 / h*(np.pi**(1/2)))**d) * (np.e**(-(delta_mag**2) / (h**2)))
    
    def dgaus_kern(self, h = 1, d = 3):
        mag_1 = self.p1.get_magnitude()
        mag_2 = self.p2.get_magnitude()
        delta_mag = mag_1 - mag_2
        return -((2*np.e**(-(delta_mag**2) / (h**2)))*(delta_mag)) / ((h**5)*(np.pi*(3/2)))

In [134]:
my_particles = particle_array(10, [0,10,0,20], 5)

In [135]:
my_particles.get_position()

[9, 4, 5]
[8, 0, 7]
[3, 0, 2]
[1, 5, 7]
[3, 6, 8]
[1, 9, 3]
[0, 3, 6]
[4, 2, 6]
[2, 1, 2]
[9, 9, 7]


In [136]:
my_particles.get_direction()

[-21.925347316223824, -21.925347316223824, -22.023447316223823]
[0.9595136768852516, 0.9595136768852516, 0.8614136768852516]
[11.86131256111777, 11.86131256111777, 11.76321256111777]
[-1.6286908984971715, -1.6286908984971715, -1.7267908984971716]
[-0.44261054196080035, -0.44261054196080035, -0.5407105419608004]
[-2.2647396288140134, -2.2647396288140134, -2.3628396288140134]
[-13.70396414191226, -13.70396414191226, -13.802064141912261]
[4.667284209673222, 4.667284209673222, 4.569184209673223]
[-11.867872993058818, -11.867872993058818, -11.965972993058818]
[30, 30, 30]


In [137]:
my_particles.get_initrho()

9.143174998232535
11.867403458197462
3.8593473173210358
4.455676523924033
11.94067718115041
7.397800942233391
3.179014260308342
4.529579328536242
3.858984336907222
3.227689922292964e-05


In [138]:
my_particles.get_rho()

9.143174998232535
11.867403458197462
3.8593473173210358
4.455676523924033
11.94067718115041
7.397800942233391
3.179014260308342
4.529579328536242
3.858984336907222
3.227689922292964e-05


In [139]:
my_particles.get_initpre()

117555.10712013258
152580.90160539592
49620.1797941276
57287.269593308985
153522.9923290767
95114.58354300074
40873.040489678686
58237.44850975168
49615.51290309285
0.41498870429480966


In [140]:
my_particles.get_pre()

117555.10712013258
152580.90160539592
49620.1797941276
57287.269593308985
153522.9923290767
95114.58354300074
40873.040489678686
58237.44850975168
49615.51290309285
0.41498870429480966


In [141]:
my_particles.get_accel()

-2192.5347316223824
95.95136768852515
1186.131256111777
-162.86908984971714
-44.26105419608003
-226.47396288140135
-1370.396414191226
466.7284209673222
-1186.7872993058818
3434.511507279065
